In [ ]:
from DQN_networks import Agent
import numpy as np
import gym
import tensorflow as tf
from utils import plotLearning
from enviroment_offloading import offloading_env
import pandas as pd
VtoM_trans = 2048  #Mbps以5G傳輸速度為範本
VtoV_trans = 3072  #Mbps
VtoC_trans = 2048  #Mbps

car_process = 300  #Mbps
MEC_process = 2000 #Mbps
if __name__ =='__main__':
    #tf.compat.v1.disable_eager_execution()#關閉tensorflow2裡的eager_execution，eager_execution這會讓tf執行的很慢
    
    #創建環境
    #env = gym.make('LunarLander-v2')
    #pig 這一段要換成自己寫的車載環境 
    #env = single_env()
    env = offloading_env(VtoM_trans,VtoV_trans,VtoC_trans,car_process,MEC_process)
    lr = 0.0001 #學習率
    n_games = 1000 #epoch
    #agent = Agent(gamma=0.99, epsilon=1.0, lr=lr, input_dims=env.observation_space.shape,
    #              n_actions = env.action_space.n, mem_size=1000000, batch_size=64, epsilon_end=0.01)
    #pig n_actions這個要換成自己從車載環境回傳的動作空間,或是直接給值,ex:直接給5(env.action_space.n 輸出為常數)
    #pig env.observation_space.shape 要換成自己觀察向量的維度，但要注意gym裡是列表型態，要設成一個值的話，DQN_networks 裡buffer裡的*要去掉。不然就是env.observation_space.shape要為列表型態
    agent = Agent(gamma=0.1, epsilon=1.0, lr=lr, input_dims=env.observation_space,
                  n_actions = env.action_space, mem_size=1000000, batch_size=64, epsilon_end=0.01)
    
    score_history = [] #紀錄得分歷史
    eps_history = [] #紀錄epsilon歷史
    best_score = -1.6#平均要超過才能保存
    for i in range(n_games): #主訓練迴圈,訓練幾圈由epoch決定
        
        done = False
        observation = env.reset()
        setp_number = -1
        car1data = 0
        car2data = 0
        MECdata = 0
        score = 0
        
        
        while not done: #done如果false, not done=not false=true=1 ,while判斷為1就等於是無限迴圈,要終止就要done的值為true,not done才會為false
            setp_number =setp_number +1
            action = agent.choose_action(observation)
            observation_, reward, done, info,car1data ,car2data ,MECdata  ,csv_latency = env.step(action,setp_number,car1data ,car2data ,MECdata)            
#             print("第",setp_number,"個action = ", action)
#             print("延遲 = ",reward)
#             print("第一台車現在的數據輛 = ",observation_[0])
#             print("第一台車之前的數據輛 = ",car1data)  
            
#             print("第二台車現在的數據輛 = ",observation_[1])
#             print("第二台車之前的數據輛 = ",car2data)
            
#             print("MEC現在的數據輛 = ",observation_[2])   
#             print("MEC之前的數據輛 = ",MECdata)
#             print("========================================")
            score += reward #這次得到的分數就是這一圈epoch的所有reward加總的值,然後迴圈一直更新,直到done為true,就是跑完這次epoch的分數
            #score = score - reward
            agent.store_transition(observation, action, reward, observation_, done) #把這個迴圈的資料(觀察,動作,獎勵,下一個觀察,終端狀態)存進replay buffer
            observation = observation_ #要在while裡更新新的狀態,不然會一直是舊得observation進行learn
            
            agent.learn() #迭代中進行TD學習
        
        eps_history.append(agent.epsilon) #把每一個epoch的epsilon記錄下來
        score = score/(env.data_time_slot)
        
        score_history.append(score) #把每一個epoch的分數記錄下來
        
        avg_score = np.mean(score_history[-100:]) #計算平均分數,為最新一百場比賽的分數加起來平均
        if score > best_score : #如果平均分數大於最佳分數
            best_score = score  #覆蓋掉最佳分數
            print("best_score loading = ",best_score)
            best_latency_csv=pd.DataFrame(data=csv_latency)
            best_latency_csv.to_csv('best_score_latency.csv',encoding='utf-8', index = False)

        print('episode: ', i, 'score %.5f' % score, 'average_score %.5f' % avg_score, 'epsilon %.2f' % agent.epsilon)
        
        
    filename = 'offloading_DQN_1.png' #生成的收斂圖名稱,有調整超參數的話可以改變圖的名稱比較好分辨
    figure_file = 'plots/' + filename
    x = [i+1 for i in range(n_games)] #x軸,i+1=1
    plotLearning(x, score_history, eps_history, figure_file) #畫圖
        


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import csv
def plotLearning(x, scores, epsilons, filename, lines=None):
    for i in range(1000):
        scores[i] =-scores[i] 
    fig=plt.figure()
    ax=fig.add_subplot(111, label="1")
    ax2=fig.add_subplot(111, label="2", frame_on=False)

    ax.plot(x, epsilons, color="C0")
    ax.set_xlabel("Game", color="C0")
    ax.set_ylabel("Epsilon", color="C0")
    ax.tick_params(axis='x', colors="C0")
    ax.tick_params(axis='y', colors="C0")

    N = len(scores)
    running_avg = np.empty(N)
    for t in range(N):
	    running_avg[t] = np.mean(scores[max(0, t-20):(t+1)])

    ax2.plot(x, running_avg, color="C1")
    #ax2.xaxis.tick_top()
    ax2.axes.get_xaxis().set_visible(False)
    ax2.yaxis.tick_right()
    #ax2.set_xlabel('x label 2', color="C1")
    ax2.set_ylabel('Score', color="C1")
    #ax2.xaxis.set_label_position('top')
    ax2.yaxis.set_label_position('right')
    #ax2.tick_params(axis='x', colors="C1")
    plt.title('DQN_avg_scores')
    ymin, ymax = plt.ylim()
    ytick_spacing = 0.1
    yticks = np.arange(ymin, ymax + ytick_spacing, ytick_spacing)
    plt.yticks(yticks)
    ax2.tick_params(axis='y', colors="C1")

    if lines is not None:
        for line in lines:
            plt.axvline(x=line)
            
    plt.savefig(filename,bbox_inches='tight')
plotLearning(x, score_history, eps_history, filename)

In [ ]:
#測試自己的維度
import numpy as np
from agent import Agent
from utils import plot_learning_curve, make_env, manage_memory
from gym import wrappers
from enviroment_offloading import offloading_env
from replay_memory import ReplayBuffer
#環境制定
VtoM_trans = 2048  #Mbps以5G傳輸速度為範本
VtoV_trans = 3072  #Mbps
VtoC_trans = 2048  #Mbps

car_process = 300  #Mbps
MEC_process = 30000 #Mbps

#datasize_car = np.random.randint(low = 100,high = 900,size = (500,3))
#data_time_slot = 500 #總共產生幾筆數據，秒為單位

if __name__ == '__main__':
    manage_memory()
    #env1 = make_env('PongNoFrameskip-v4')
    #dims = env1.observation_space
    
    env = offloading_env(VtoM_trans,VtoV_trans,VtoC_trans,car_process,MEC_process)
    best_score = -np.inf
    load_checkpoint = False
    record_agent = False
    n_games = 200
    
    
    dims = np.zeros([3],dtype = "float32")

    #dims = np.load('input_dims.npy',allow_pickle=True)
    #print("input_dims = ",dims)
    #print("input_dims shape = ",dims.shape)
    
    
    agent = Agent(gamma=0.99, epsilon=1, lr=0.0001,
                  input_dims = dims.shape ,n_actions=4, 
                  mem_size=50000, eps_min=0.1,
                  batch_size=32, replace=1000, eps_dec=1e-5,
                  chkpt_dir='models/', algo='DQNAgent',
                  env_name='enviroment_offloading')
    if load_checkpoint:
        agent.load_models()
        agent.epsilon = agent.eps_min

    fname = agent.algo + '_' + agent.env_name + '_lr' + str(agent.lr) + '_' \
        + str(n_games) + 'games'
    figure_file = 'plots/' + fname + '.png'
    # if you want to record video of your agent playing, do a
    # mkdir video
    n_steps = 0
    scores, eps_history, steps_array = [], [], []

    for i in range(n_games):
        done = False
        observation = env.reset()
        setp_number = 0
        car1data = 0
        car2data = 0
        MECdata = 0
        score = 0
        
        while not done:
           
            setp_number =setp_number +1
            action = agent.choose_action(observation)
            observation_, reward, done, info,car1data ,car2data ,MECdata  ,total_latency = env.step(action,setp_number,car1data ,car2data ,MECdata)
#             print("第",setp_number,"個action = ", action)
#             print("延遲 = ",reward)
#             print("第一台車現在的數據輛 = ",observation_[0])
#             print("第一台車之前的數據輛 = ",car1data)  
            
#             print("第二台車現在的數據輛 = ",observation_[1])
#             print("第二台車之前的數據輛 = ",car2data)
            
#             print("MEC現在的數據輛 = ",observation_[2])   
#             print("MEC之前的數據輛 = ",MECdata)
#             print("========================================")
            score = score - reward
            if not load_checkpoint:
                agent.store_transition(observation, action,reward, observation_, done)
                agent.learn()
            observation = observation_
            n_steps += 1
        
        #source = score/setp_number
        #print (setp_number)
        #print (score)
        score = score/(env.data_time_slot)
        print (score)
        scores.append(score)
        steps_array.append(n_steps)
        
        avg_score = np.mean(scores[-50:])
        print('episode {} score {:.4f} avg score {:.4f} '
              'best score {:.4f} epsilon {:.2f} steps {}'.
              format(i, score, avg_score, best_score, agent.epsilon,
                     n_steps))

        if score > best_score:
            #if not load_checkpoint:
                #agent.save_models()
            best_score = score
            np.savetxt('latency_time.csv',total_latency, delimiter=",")

        eps_history.append(agent.epsilon)

    x = [i+1 for i in range(len(scores))]
    plot_learning_curve(steps_array, scores, eps_history, figure_file)
